# Clustering Crypto

In [269]:
# Initial imports
import requests
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [270]:
print(f"Current working directory:{Path.cwd()}")

Current working directory:/Users/aalfiler/Sites/hw_AWS/AWS/Code


In [271]:
# Alternatively, use the provided csv file:
file_path = Path("/Users/aalfiler/Sites/hw_AWS/AWS/Code/Resources/crypto_data.csv")
crypto_df = pd.read_csv(file_path)
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [272]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df.drop(columns=["Unnamed: 0"])
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [273]:
# Keep only cryptocurrencies that are trading
crypto_df.drop(crypto_df[crypto_df['IsTrading'] == False].index, inplace=True)
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
1243,Super Zero,Ethash,True,PoW,NaN,1000000000
1244,UOS,SHA-256,True,DPoI,NaN,1000000000
1245,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [274]:
crypto_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1144 entries, 0 to 1247
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CoinName         1144 non-null   object 
 1   Algorithm        1144 non-null   object 
 2   IsTrading        1144 non-null   bool   
 3   ProofType        1144 non-null   object 
 4   TotalCoinsMined  685 non-null    float64
 5   TotalCoinSupply  1144 non-null   object 
dtypes: bool(1), float64(1), object(4)
memory usage: 54.7+ KB


In [281]:
crypto_df["TotalCoinSupply"] = crypto_df["TotalCoinSupply"].str.replace(" ", "")
crypto_df["TotalCoinSupply"] = crypto_df["TotalCoinSupply"].str.replace("10.500.000", "10500000")
crypto_df["TotalCoinSupply"] = crypto_df["TotalCoinSupply"].str.replace("1.000.000", "1000000")

In [282]:
crypto_df["TotalCoinSupply"] = crypto_df.TotalCoinSupply.astype(float)

In [283]:
crypto_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1144 entries, 0 to 1247
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CoinName         1144 non-null   object 
 1   Algorithm        1144 non-null   object 
 2   IsTrading        1144 non-null   bool   
 3   ProofType        1144 non-null   object 
 4   TotalCoinsMined  685 non-null    float64
 5   TotalCoinSupply  1144 non-null   float64
dtypes: bool(1), float64(2), object(3)
memory usage: 54.7+ KB


In [284]:
IsTrading_check = crypto_df['IsTrading'].unique()
IsTrading_check

array([ True])

In [285]:
# Remove the "IsTrading" column
crypto_df = crypto_df.drop(columns=["IsTrading"])
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01
1,365Coin,X11,PoW/PoS,NaN,2.300000e+09
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08
3,SixEleven,SHA-256,PoW,NaN,6.110000e+05
4,808,SHA-256,PoW/PoS,0.000000e+00,0.000000e+00


In [286]:
crypto_df.shape

(1144, 5)

In [287]:
crypto_df.isnull().sum()

CoinName             0
Algorithm            0
ProofType            0
TotalCoinsMined    459
TotalCoinSupply      0
dtype: int64

In [288]:
# Remove rows with at least 1 null value
crypto_df = crypto_df.dropna().copy()

In [289]:
crypto_df.isnull().sum()

CoinName           0
Algorithm          0
ProofType          0
TotalCoinsMined    0
TotalCoinSupply    0
dtype: int64

In [290]:
crypto_df.shape

(685, 5)

In [291]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df.drop(crypto_df[crypto_df['TotalCoinsMined'] == 0].index, inplace=True)
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08
5,EliteCoin,X13,PoW/PoS,2.927942e+10,3.141593e+11
7,Bitcoin,SHA-256,PoW,1.792718e+07,2.100000e+07
8,Ethereum,Ethash,PoW,1.076842e+08,0.000000e+00
...,...,...,...,...,...
1238,ZEPHYR,SHA-256,DPoS,2.000000e+09,2.000000e+09
1242,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,2.500000e+08
1245,Beldex,CryptoNight,PoW,9.802226e+08,1.400223e+09
1246,Horizen,Equihash,PoW,7.296538e+06,2.100000e+07


In [292]:
# Drop rows where there are 'N/A' text values --- need verify
crypto_df.loc[~(crypto_df=='N/A').all(axis=1)]

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08
5,EliteCoin,X13,PoW/PoS,2.927942e+10,3.141593e+11
7,Bitcoin,SHA-256,PoW,1.792718e+07,2.100000e+07
8,Ethereum,Ethash,PoW,1.076842e+08,0.000000e+00
...,...,...,...,...,...
1238,ZEPHYR,SHA-256,DPoS,2.000000e+09,2.000000e+09
1242,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,2.500000e+08
1245,Beldex,CryptoNight,PoW,9.802226e+08,1.400223e+09
1246,Horizen,Equihash,PoW,7.296538e+06,2.100000e+07


In [293]:
crypto_df = crypto_df.reset_index()
crypto_df.head()

,index,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01
1,2,404Coin,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08
2,5,EliteCoin,X13,PoW/PoS,2.927942e+10,3.141593e+11
3,7,Bitcoin,SHA-256,PoW,1.792718e+07,2.100000e+07
4,8,Ethereum,Ethash,PoW,1.076842e+08,0.000000e+00


In [294]:
crypto_df.drop(columns=['index'], inplace=True)

In [295]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
CoinName_df = crypto_df.drop(columns=["Algorithm", "ProofType", "TotalCoinsMined", "TotalCoinSupply"])
CoinName_df.head()

,CoinName
0,42 Coin
1,404Coin
2,EliteCoin
3,Bitcoin
4,Ethereum


In [296]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df2 = crypto_df.drop(columns="CoinName")
crypto_df2

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01
1,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08
2,X13,PoW/PoS,2.927942e+10,3.141593e+11
3,SHA-256,PoW,1.792718e+07,2.100000e+07
4,Ethash,PoW,1.076842e+08,0.000000e+00
...,...,...,...,...
528,SHA-256,DPoS,2.000000e+09,2.000000e+09
529,Scrypt,PoW/PoS,1.493105e+07,2.500000e+08
530,CryptoNight,PoW,9.802226e+08,1.400223e+09
531,Equihash,PoW,7.296538e+06,2.100000e+07


In [297]:
# Create dummy variables for text features
crypto_df2_encoded = pd.get_dummies(crypto_df2, columns=["Algorithm", "ProofType", "TotalCoinsMined", "TotalCoinSupply"])
crypto_df2_encoded.head()

,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,Algorithm_Cloverhash,Algorithm_Counterparty,...,TotalCoinSupply_90600000000.0,TotalCoinSupply_92000000000.0,TotalCoinSupply_99000000000.0,TotalCoinSupply_184000000000.0,TotalCoinSupply_184467440735.0,TotalCoinSupply_184470000000.0,TotalCoinSupply_200000000000.0,TotalCoinSupply_314159265359.0,TotalCoinSupply_384000000000.0,TotalCoinSupply_990000000000.0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [298]:
# Standardize data
data_scaler = StandardScaler()

In [299]:
data_scaler.fit(crypto_df2_encoded)

StandardScaler()

In [300]:
crypto_df2_scaled = data_scaler.transform(crypto_df2_encoded)
crypto_df2_scaled[:5]

array([[-0.0433555 , -0.0433555 , -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.0433555 , -0.0433555 , -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.0433555 , -0.0433555 , -0.0433555 , ..., 23.06512519,
        -0.0433555 , -0.0433555 ],
       [-0.0433555 , -0.0433555 , -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.0433555 , -0.0433555 , -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ]])

In [301]:
crypto_df2_path = Path("/Users/aalfiler/Sites/hw_AWS/AWS/Code/Resources/crypto_data_encoded.csv") # store scaled data instead?
crypto_df2_encoded.to_csv(crypto_df2_path, index=False)

### Reducing Dimensions Using PCA

In [302]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_df2_pca = pca.fit_transform(crypto_df2_scaled)

In [303]:
# Create a DataFrame with the principal components data
crypto_df2_pca = pd.DataFrame(
    data=crypto_df2_pca, columns=["principal component 1", "principal component 2", 'principal component 3']
)
crypto_df2_pca.head()

,principal component 1,principal component 2,principal component 3
0,-0.045174,-0.249515,0.130105
1,0.008459,-0.244397,0.050598
2,-0.030812,-0.147526,0.112786
3,-0.134877,-0.214711,-0.081201
4,-0.235036,0.003539,0.049404


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [304]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_df2_pca)
    inertia.append(km.inertia_)

In [305]:
# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [306]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)
# Fit the model
model.fit(crypto_df2_pca)
# Predict clusters
predictions = model.predict(crypto_df2_pca)

crypto_df2_pca["class"] = model.labels_

# Create a new DataFrame including predicted clusters and cryptocurrencies features -- ERRORRRRRRRRR
clustered_df = pd.concat([crypto_df2, crypto_df2_pca, CoinName_df], axis='columns', join='inner')
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,principal component 1,principal component 2,principal component 3,class,CoinName
0,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.045174,-0.249515,0.130105,0,42 Coin
1,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,0.008459,-0.244397,0.050598,0,404Coin
2,X13,PoW/PoS,2.927942e+10,3.141593e+11,-0.030812,-0.147526,0.112786,0,EliteCoin
3,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.134877,-0.214711,-0.081201,0,Bitcoin
4,Ethash,PoW,1.076842e+08,0.000000e+00,-0.235036,0.003539,0.049404,0,Ethereum


In [307]:
clustered_df.set_index(clustered_df['CoinName'], inplace=True)
clustered_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,principal component 1,principal component 2,principal component 3,class,CoinName
CoinName,,,,,,,,,
42 Coin,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.045174,-0.249515,0.130105,0,42 Coin
404Coin,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,0.008459,-0.244397,0.050598,0,404Coin
EliteCoin,X13,PoW/PoS,2.927942e+10,3.141593e+11,-0.030812,-0.147526,0.112786,0,EliteCoin
Bitcoin,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.134877,-0.214711,-0.081201,0,Bitcoin
Ethereum,Ethash,PoW,1.076842e+08,0.000000e+00,-0.235036,0.003539,0.049404,0,Ethereum
...,...,...,...,...,...,...,...,...,...
ZEPHYR,SHA-256,DPoS,2.000000e+09,2.000000e+09,0.177115,0.077931,0.359122,0,ZEPHYR
Gapcoin,Scrypt,PoW/PoS,1.493105e+07,2.500000e+08,-0.043726,-0.199523,0.033251,0,Gapcoin
Beldex,CryptoNight,PoW,9.802226e+08,1.400223e+09,-0.061149,-0.212308,-0.120765,0,Beldex


### Visualizing Results

#### 3D-Scatter with Clusters

In [308]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    crypto_df2_pca,
    x="principal component 3",
    y="principal component 2",
    z="principal component 1",
    color="class",
    symbol="class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Table of Tradable Cryptocurrencies

In [309]:
# Table with tradable cryptos
tradable_cryptos = pd.DataFrame(data=clustered_df, columns=['TotalCoinsMined', 'TotalCoinSupply'])
tradable_cryptos

,TotalCoinsMined,TotalCoinSupply
CoinName,,
42 Coin,4.199995e+01,4.200000e+01
404Coin,1.055185e+09,5.320000e+08
EliteCoin,2.927942e+10,3.141593e+11
Bitcoin,1.792718e+07,2.100000e+07
Ethereum,1.076842e+08,0.000000e+00
...,...,...
ZEPHYR,2.000000e+09,2.000000e+09
Gapcoin,1.493105e+07,2.500000e+08
Beldex,9.802226e+08,1.400223e+09


In [310]:
# Print the total number of tradable cryptocurrencies
tradable_cryptos.nunique()

TotalCoinsMined    519
TotalCoinSupply    260
dtype: int64

#### Scatter Plot with Tradable Cryptocurrencies

In [311]:
# Scale data to create the scatter plot
tradable_cryptos.describe()

,TotalCoinsMined,TotalCoinSupply
count,5.330000e+02,5.330000e+02
mean,5.319333e+09,7.765903e+09
std,4.560537e+10,5.201320e+10
min,-5.917978e+09,0.000000e+00
25%,8.305775e+06,1.000000e+07
50%,2.448794e+07,3.000000e+07
75%,1.835340e+08,2.100000e+08
max,9.899887e+11,9.900000e+11


In [312]:
tradable_cryptos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 533 entries, 42 Coin to BitcoinPlus
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   TotalCoinsMined  533 non-null    float64
 1   TotalCoinSupply  533 non-null    float64
dtypes: float64(2)
memory usage: 12.5+ KB


In [321]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
tradable_cryptos.hvplot.scatter(
    title="Crypto Supply",
    x="TotalCoinsMined", 
    y="TotalCoinSupply", 
    xlabel="Total Coins Mined", 
    ylabel="Total Coins Supply", 
    hover_cols=["CoinName"], 
    width=1000,
    height=650,
    logy=True,
    logx=True
)

:Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)